In [2]:
%cd ..

/Users/mateoibarguen/Desktop/DATA-Capstone


In [3]:
from models.networks.lstm_models import *
from models.spectra_preprocessor import *

In [21]:
spectra_pp = SpectraPreprocessor(dataset_name='test', num_channels=8, num_instances=100, use_generator=False)

In [22]:
X_train, y_train, X_test, y_test = spectra_pp.transform(encoded=True)

In [25]:
X_train.shape

(85, 301, 8)